Example usage of FlowClient to run daily location, modal location and flow queries.

In [ ]:
import flowclient

## Create an access token
In actual usage, this would be done using [FlowAuth](../../flowauth).

In [ ]:
import os
import sys
from dotenv import find_dotenv

In [ ]:
path_to_utils_module = os.path.join(
    os.path.dirname(find_dotenv()), "..", "integration_tests", "tests"
)
sys.path.insert(0, path_to_utils_module)

In [ ]:
from datetime import timedelta
from utils import make_token

In [ ]:
claims = {
    "daily_location": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2"],
    },
    "modal_location": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2"],
    },
    "flows": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2", "admin1"],
    },
}

In [ ]:
TOKEN = make_token(
    username="testuser",
    secret_key="secret",
    lifetime=timedelta(days=1),
    claims=claims,
)

## Create an API connection

In [ ]:
conn = flowclient.connect("http://localhost:9090", TOKEN)

## Daily location

### Specify query parameters

In [ ]:
query_spec = flowclient.daily_location(
    date="2016-01-01",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)

query_spec

### Set query running

In [ ]:
query_id = flowclient.run_query(conn, query_spec)
query_id

### Check status of query

In [ ]:
flowclient.get_status(conn, query_id)

### Get query result

In [ ]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

Alternative (run query, wait for it to finish, and get result, in one function call):

In [ ]:
df = flowclient.get_result(conn, query_spec)
df.head()

## Modal location
### Specify query parameters
Either explicitly from a set of daily locations:

In [ ]:
dl1 = flowclient.daily_location(
    date="2016-01-01",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)
dl2 = flowclient.daily_location(
    date="2016-01-02",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)
ml = flowclient.modal_location(dl1, dl2, aggregation_unit="admin3")

ml

Or from a date range:

In [ ]:
ml = flowclient.modal_location_from_dates(
    "2016-01-01",
    "2016-01-03",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)

ml

### Set query running

In [ ]:
query_id = flowclient.run_query(conn, ml)
query_id

### Check status of query

In [ ]:
flowclient.get_status(conn, query_id)

### Get query result

In [ ]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

Alternative:

In [ ]:
df = flowclient.get_result(conn, ml)
df.head()

## Flow
### Specify query parameters
Specify parameters for two location queries:

In [ ]:
aggregation_unit = "admin1"
dl_past = flowclient.daily_location(
    "2016-01-01",
    aggregation_unit=aggregation_unit,
    daily_location_method="last",
    subscriber_subset=None,
)
dl_present = flowclient.daily_location(
    "2016-01-07",
    aggregation_unit=aggregation_unit,
    daily_location_method="last",
    subscriber_subset=None,
)

Specify parameters for flow query:

In [ ]:
flow = flowclient.flows(dl_past, dl_present, aggregation_unit=aggregation_unit)
flow

### Set query running

In [ ]:
query_id = flowclient.run_query(conn, flow)
query_id

### Check status of query

In [ ]:
flowclient.get_status(conn, query_id)

### Get query result

In [ ]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()